# IBM Data Science Professional Certificate
## Capstone Project: Recommendation of London Boroughs
### Data Modelling and Analysis

This notebook is the second out of two that are used to build a London Borough recommender system. Is is quite natural to split the process in two parts:  
1. Data Exploration and Preparation
2. __Data Modelling and Analysis__

The general approach is as follows: both notebooks are going to be used for illustration purposes with all the code and comments displayed in code cells. I am not going to create functions to remove as much abstraction as possible. However, I intend to wrap all of this work into a single web application that will be shared as the final product with interactive features.

This notebook deals with data modelling, analysis, and its application to create a recommendation

#### Data Sources

The following data sources are needed to recommend Greater London boroughs to the user:  

1. Rent data - available from London Datastore in *.xls format: [Average Private Rents, Borough](https://data.london.gov.uk/dataset/average-private-rents-borough)
2. List of London Boroughs and their geographical coordinates, available from Wikipedia (html table): [List of London Boroughs](https://en.wikipedia.org/wiki/List_of_London_boroughs)
3. Greater London borough boundaries in GeoJSON format, available [here](https://joshuaboyd1.carto.com/tables/london_boroughs_proper/public)
4. Information on venues in Greater London area. Available via free version of [Foursquare API](https://developer.foursquare.com/).

#### Methodology

This section describes the methodology of the recommendation algorithm. The algorithm for this system is rather simple, but this is mainly due to the fact that we do not have a large sample of user data and are relying on explicit data collection from a single user.

##### Calculation of the recommended boroughs

The purpose of the algorithm is to recommend a list of boroughs to a user based on their specified preferences. 

Let $W$ be a venue matrix of dimensions $n \times m$, where every element represents a normalized density of venue group $j$ in borough $i$:

$$W = \begin{bmatrix} a_{11} & ... & a_{1m} \\ ... & a_{ij} & ... \\ a_{n1} & ... & a_{nm} \end{bmatrix}$$

The user-provided preferences can be formally written as a column vector $p$:

$$p = \begin{bmatrix} p_{1} \\ ... \\ p_{m} \end{bmatrix}$$

Recommendation matrix $R$ then can be calculated as a matrix product:

$$R = W \times p  = \begin{bmatrix} r_{1} \\ ... \\ r_{m} \end{bmatrix}$$

This matrix is a column vector of length $n$ where each row represents the weight of each borough. The larger the weight, the closer the borough matches users preference. 

Given a number $N$ of boroughs we want to recommend, $N$ largest weights are selected and the list of boroughs are returned in an order from the highest weigh $r$ to the lowest.


##### Filtering venue matrix based

The venue density matrix $W$ is filtered in advance based on user's selection on upper and lower limits for rent: $m_{min}$ and $m_{max}$ and the accommodation type. The dataset for borough rent data contains first $q_{1}$ and third quartile $q_{3}$ information. Only the boroughs satisfying either of the below condition are kept in the $W$ matrix:
    
$$ \begin{align*}
  q_{1} \le m_{max} \land q_{3} \ge m_{max} \\ 
  q_{1} \le m_{min} \land q_{3} \ge m_{min} \\ 
\end{align*}
$$

##### Library Imports

In [1]:
import pandas as pd
import numpy as np
import folium

##### Reading in the pre-processed files

In [140]:
df_venues_raw = pd.read_csv('london_venues.csv')
df_rent_raw = pd.read_csv('london_boroughs.csv')
ldn_geojson = 'london_boroughs_proper.geojson'

In [141]:
df_venues_raw.head()

,Venue,Borough,Venue Category,Group,Venue Latitude,Venue Longitude,BoroughLocation
0,Southbank Centre,Lambeth,Performing Arts Venue,Entertainment,51.505843,-0.116985,"[51.4607, -0.1163]"
1,Whitehall Gardens,Westminster,Garden,Green spaces,51.506354,-0.122900,"[51.4973, -0.1372]"
2,Gordon's Wine Bar,Westminster,Wine Bar,Going out,51.507911,-0.123293,"[51.4973, -0.1372]"
3,National Theatre,Lambeth,Theater,Entertainment,51.507376,-0.114793,"[51.4607, -0.1163]"
4,Royal Festival Hall,Lambeth,Concert Hall,Entertainment,51.505838,-0.116667,"[51.4607, -0.1163]"


In [142]:
df_venues_raw['BoroughLat'] = df_venues_raw['BoroughLocation'].apply(lambda x: float(x.split(',')[0][1:]))
df_venues_raw['BoroughLon'] = df_venues_raw['BoroughLocation'].apply(lambda x: float(x.split(',')[1][:-1]))                                                                 

In [136]:
df_venues_raw.head()

,Venue,Borough,Venue Category,Group,Venue Latitude,Venue Longitude,BoroughLocation,BoroughLat,BoroughLon
0,Southbank Centre,Lambeth,Performing Arts Venue,Entertainment,51.505843,-0.116985,"[51.4607, -0.1163]",51.4607,-0.1163
1,Whitehall Gardens,Westminster,Garden,Green spaces,51.506354,-0.122900,"[51.4973, -0.1372]",51.4973,-0.1372
2,Gordon's Wine Bar,Westminster,Wine Bar,Going out,51.507911,-0.123293,"[51.4973, -0.1372]",51.4973,-0.1372
3,National Theatre,Lambeth,Theater,Entertainment,51.507376,-0.114793,"[51.4607, -0.1163]",51.4607,-0.1163
4,Royal Festival Hall,Lambeth,Concert Hall,Entertainment,51.505838,-0.116667,"[51.4607, -0.1163]",51.4607,-0.1163


In [69]:
df_venues_raw.shape

(15847, 7)

In [70]:
df_rent_raw.head()

,Year,Quarter,Code,Area,Category,Count of rents,Average,Lower quartile,Median,Upper quartile
0,2011,Q2,E09000001,City of London,Room,-,-,-,-,-
1,2011,Q2,E09000002,Barking and Dagenham,Room,92,336,282,347,390
2,2011,Q2,E09000003,Barnet,Room,945,450,399,433,500
3,2011,Q2,E09000004,Bexley,Room,119,390,347,390,433
4,2011,Q2,E09000005,Brent,Room,344,469,390,457,550


##### Preparation of rent data:

In [71]:
# 1. Keep Q1 2019 only
df_rent = df_rent_raw.loc[(df_rent_raw['Year']==2019) & (df_rent_raw['Quarter']=='Q1')]
    
# 2. Following regions are excluded (not boroughs)
exclude = [
     'NORTH EAST',
     'NORTH WEST',
     'YORKSHIRE AND THE HUMBER',
     'EAST MIDLANDS',
     'WEST MIDLANDS',
     'EAST',
     'LONDON',
     'SOUTH EAST',
     'SOUTH WEST',
     'Inner London',
     'Outer London'
]

df_rent = df_rent.loc[~df_rent['Area'].isin(exclude)]

# 3. Rename 'Area' to 'Borough'                                      
df_rent.rename(columns={'Area': 'Borough'}, inplace=True)
                                                 
# 4. Keep relevant columns                                         
keep_cols = ['Borough', 'Category', 'Lower quartile', 'Median', 'Upper quartile']
df_rent = df_rent[keep_cols]
for col in ['Lower quartile', 'Median', 'Upper quartile']:
    df_rent[col] = pd.to_numeric(df_rent[col], errors='coerce')

df_rent.head()

,Borough,Category,Lower quartile,Median,Upper quartile
5852,City of London,Room,NaN,NaN,NaN
5853,Barking and Dagenham,Room,600.0,650.0,700.0
5854,Barnet,Room,550.0,588.0,650.0
5855,Bexley,Room,550.0,585.0,672.0
5856,Brent,Room,500.0,602.0,700.0


##### Preparation of venues data:

For venues data, we need to transform the dataframe so that we have:  
1. Venue categories as column names
2. Borough names as index (i.e. row 'names')
3. Values representing normalized density per borrow (i.e. percentage of each venue category within the borough)

In [72]:
df_venues = df_venues_raw.copy()

# 1. & 2. Move venue categories to column names by using one-hot encoding
# One-hot encoding
df_oh = pd.get_dummies(df_venues['Group'])
df_venues = pd.concat([df_venues, df_oh], axis=1)
group_cols = df_oh.columns.tolist()

df_venues.head()

,Venue,Borough,Venue Category,Group,Venue Latitude,Venue Longitude,BoroughLocation,Eating out,Entertainment,Going out,Green spaces,Groceries,Health and Sports,Other,Public Transport,Shopping
0,Southbank Centre,Lambeth,Performing Arts Venue,Entertainment,51.505843,-0.116985,"[51.4607, -0.1163]",0,1,0,0,0,0,0,0,0
1,Whitehall Gardens,Westminster,Garden,Green spaces,51.506354,-0.122900,"[51.4973, -0.1372]",0,0,0,1,0,0,0,0,0
2,Gordon's Wine Bar,Westminster,Wine Bar,Going out,51.507911,-0.123293,"[51.4973, -0.1372]",0,0,1,0,0,0,0,0,0
3,National Theatre,Lambeth,Theater,Entertainment,51.507376,-0.114793,"[51.4607, -0.1163]",0,1,0,0,0,0,0,0,0
4,Royal Festival Hall,Lambeth,Concert Hall,Entertainment,51.505838,-0.116667,"[51.4607, -0.1163]",0,1,0,0,0,0,0,0,0


In [73]:
# 3. Sum all the venues and create a 'Total' column
df_groups = df_venues.groupby(['Borough'], as_index=False)[group_cols].sum()
df_groups['Total'] = df_groups[group_cols].sum(axis=1)

df_groups.head()

,Borough,Eating out,Entertainment,Going out,Green spaces,Groceries,Health and Sports,Other,Public Transport,Shopping,Total
0,Barking and Dagenham,31.0,9.0,9.0,14.0,14.0,12.0,25.0,11.0,28.0,153.0
1,Barnet,218.0,12.0,43.0,44.0,40.0,40.0,51.0,24.0,174.0,646.0
2,Bexley,64.0,6.0,32.0,9.0,21.0,14.0,26.0,12.0,79.0,263.0
3,Brent,170.0,11.0,41.0,27.0,29.0,21.0,40.0,29.0,131.0,499.0
4,Bromley,130.0,15.0,67.0,45.0,27.0,37.0,58.0,29.0,116.0,524.0


In [74]:
# Normalize (i.e. so that it sums to 1)
df_groups_norm = df_groups.copy()
df_groups_norm[group_cols] = df_groups[group_cols].divide(df_groups['Total'], axis=0)

# Remove 'Total'
keep_cols = df_groups_norm.columns.tolist()[:-1]
df_groups_norm = df_groups_norm[keep_cols]  

df_groups_norm.head()

,Borough,Eating out,Entertainment,Going out,Green spaces,Groceries,Health and Sports,Other,Public Transport,Shopping
0,Barking and Dagenham,0.202614,0.058824,0.058824,0.091503,0.091503,0.078431,0.163399,0.071895,0.183007
1,Barnet,0.337461,0.018576,0.066563,0.068111,0.061920,0.061920,0.078947,0.037152,0.269350
2,Bexley,0.243346,0.022814,0.121673,0.034221,0.079848,0.053232,0.098859,0.045627,0.300380
3,Brent,0.340681,0.022044,0.082164,0.054108,0.058116,0.042084,0.080160,0.058116,0.262525
4,Bromley,0.248092,0.028626,0.127863,0.085878,0.051527,0.070611,0.110687,0.055344,0.221374


#### Creating a recommendation 

We are assuming that he user is going to provide us with the following inputs:  
1. Rent range: `rent_min` and `rent_max`
2. A list of accommodation categories: 'One Bedroom', 'Studio', 'Room', ...
3. A ranked list of preferred venue groups: 1. Entertainment, 2. Going out, 3. Public transport,...

The algorithm is implemented in two steps:  

1. The inputs from 1 and 2 are going to be used to reduce the number of boroughs that are elligible - no reason to recommend a borough that user cannot afford. 
2. Filtered `df_groups_norm` is then used to recommend five* closest matching boroughs.

_*This parameter can be changed_

Filtering of rent data is implemented in the function `filter_rent_data`:

In [40]:
def filter_rent_data(df=df_rent, categories=None, rent_range=None):
    """
    Returns boroughs that satisfy the conditions for `categories` and `rent_range`
        
    Inputs:
        df - pandas DataFrame containing rent data -> !!! Assumes df_rent as default 
        categories - an iterable or a string specifying appropriate accommodation types
        rent_range - a list or a tuple with r_min and r_max rent ranges.
        
    Output:
        boroughs - a list of boroughs that match the condition
    
    """
    if isinstance(categories, str):
        cats = [categories]
    else:
        cats = categories
    
    cat_cond = df['Category'].isin(cats)
    rent_lower = rent_range[0]
    rent_higher = rent_range[1]
    
    # If invalid data provided
    if rent_lower > rent_higher:
        rent_higher = rent_lower
    
    rent_cond_1 = (df['Lower quartile'] <= rent_higher) & (df['Upper quartile'] >= rent_higher)
    rent_cond_2 = (df['Lower quartile'] <= rent_lower) & (df['Upper quartile'] >= rent_lower)
    rent_cond = rent_cond_1 | rent_cond_2
    rent_cond = (df['Lower quartile'] <= rent_higher) & (df['Upper quartile'] >= rent_lower)
    
    not_null = ~df['Median'].isnull()

    df_filtered = df.loc[(cat_cond & rent_cond & not_null)]
    
    boroughs = df_filtered['Borough'].unique().tolist()
   
    return boroughs

The `recommend_boroughs` function performs the recommendation of top five boroughs given user's preference vector as a list of ranked venue types:

In [56]:
def recommend_boroughs(W=df_groups_norm, p=None, n_brghs=5):
    """
    Calculates matrix product of DataFrames W and p
    
    Inputs:
        W - pandas DataFrame containing venue group density for each borough
        p - pandas DataFrame with user group preferences
        n_brghs - int, number of boroughs to recommend
    
    Returns:
        rec_boroughs - `n_brghs` number of boroughs by highest match value
        df_rec - resulting recommendation matrix as pandas DataFrame
    
    """
    W_ = W.copy()
    W_.set_index(['Borough'], inplace=True)
    df_rec = (p['Preference'] * W_).sum(axis=1).to_frame()
    df_rec.columns = ['Match']
    df_rec.sort_values(by='Match', ascending=False, inplace=True)
    rec_boroughs = df_rec.head(n_brghs).index.tolist()
    return rec_boroughs, df_rec


#### Recommending boroughs given user preferences

First, let's define user inputs and their ranges:  
1. `rent_range` - a list (or tuple) with [rent_min, rent_max]. Ranges: 0 - 3000
2. `acm_type` - a list (or tuple) or a string specifying preferred acommodation type. Allowed calues listed belw in `available_acms`
3. `venue_group_rank` - a list (or tuple) with ranked venue groups. Allowed values listed out below in `available_groups`


In [75]:
available_acms = df_rent['Category'].unique().tolist()
available_acms

['Room',
 'Studio',
 'One Bedroom',
 'Two Bedroom',
 'Three Bedroom',
 'Four or more Bedrooms',
 'All categories']

In [76]:
available_groups = df_groups_norm.columns.tolist()[1:]
available_groups

['Eating out',
 'Entertainment',
 'Going out',
 'Green spaces',
 'Groceries',
 'Health and Sports',
 'Other',
 'Public Transport',
 'Shopping']

##### Preferred rent range and accommodation type:

In [77]:
rent_range = [800, 1200]
acm_type = ['Studio', 'One Bedroom']

In [78]:
available_boroughs = filter_rent_data(categories=acm_type, rent_range=rent_range)

available_boroughs

['Barnet',
 'Brent',
 'Bromley',
 'Camden',
 'Ealing',
 'Enfield',
 'Greenwich',
 'Hackney',
 'Hammersmith and Fulham',
 'Haringey',
 'Harrow',
 'Hillingdon',
 'Hounslow',
 'Islington',
 'Kensington and Chelsea',
 'Kingston upon Thames',
 'Lambeth',
 'Lewisham',
 'Merton',
 'Newham',
 'Redbridge',
 'Richmond upon Thames',
 'Southwark',
 'Sutton',
 'Tower Hamlets',
 'Waltham Forest',
 'Wandsworth',
 'Barking and Dagenham',
 'Bexley',
 'Croydon',
 'Havering']

In [79]:
df_groups_filtered = df_groups_norm.loc[df_groups_norm['Borough'].isin(available_boroughs)]
df_groups_filtered.head()

,Borough,Eating out,Entertainment,Going out,Green spaces,Groceries,Health and Sports,Other,Public Transport,Shopping
0,Barking and Dagenham,0.202614,0.058824,0.058824,0.091503,0.091503,0.078431,0.163399,0.071895,0.183007
1,Barnet,0.337461,0.018576,0.066563,0.068111,0.061920,0.061920,0.078947,0.037152,0.269350
2,Bexley,0.243346,0.022814,0.121673,0.034221,0.079848,0.053232,0.098859,0.045627,0.300380
3,Brent,0.340681,0.022044,0.082164,0.054108,0.058116,0.042084,0.080160,0.058116,0.262525
4,Bromley,0.248092,0.028626,0.127863,0.085878,0.051527,0.070611,0.110687,0.055344,0.221374


##### Ranked venue group preferences.

For convenience a `create_preferences` function is created to easily convert a list of preferred venue groups to preference vector in pandas DataFrame format

In [80]:
def create_preferences(ranking=None):
    "Converts `ranking` list to normalized pandas DataFrame"
    pref_dict = {
        'Eating out': 0,
        'Entertainment': 0,
        'Green spaces': 0,
        'Groceries': 0,
        'Health and Sports': 0,
        'Nightlife': 0,
        'Other': 0,
        'Public Transport': 0,
        'Shopping': 0,
    }
    
    N = len(ranking)
    
    for i, cat in enumerate(ranking): 
        pref_dict[cat] = N - i
    
    df = pd.DataFrame.from_dict(pref_dict, orient='index', columns=['Preference'])
    df['Preference'] = df['Preference'] / df['Preference'].sum()
    
    return df

In [81]:
venue_group_rank = [
    'Public Transport', # 1
    'Groceries',        # 2
    'Green spaces'      # 3 
]

p = create_preferences(ranking=venue_group_rank)

p

,Preference
Eating out,0.000000
Entertainment,0.000000
Green spaces,0.166667
Groceries,0.333333
Health and Sports,0.000000
Nightlife,0.000000
Other,0.000000
Public Transport,0.500000
Shopping,0.000000


##### Recommended boroughs:

In [82]:
rec_boroughs, df_rec = recommend_boroughs(W=df_groups_filtered, p=p)

In [83]:
rec_boroughs

['Barking and Dagenham', 'Sutton', 'Merton', 'Enfield', 'Havering']

In [84]:
df_rec['Match']

Borough
Barking and Dagenham      0.081699
Sutton                    0.073892
Merton                    0.064701
Enfield                   0.062024
Havering                  0.061616
Redbridge                 0.060865
Haringey                  0.060754
Greenwich                 0.060327
Bromley                   0.059160
Newham                    0.058974
Lewisham                  0.057495
Brent                     0.057448
Ealing                    0.056258
Kingston upon Thames      0.056000
Croydon                   0.055556
Bexley                    0.055133
Richmond upon Thames      0.054497
Hounslow                  0.053237
Harrow                    0.051941
Barnet                    0.050568
Hillingdon                0.044531
Waltham Forest            0.044381
Wandsworth                0.041491
Camden                    0.040496
Tower Hamlets             0.039414
Lambeth                   0.038778
Hackney                   0.038596
Southwark                 0.037037
Kensington a

### Putting it all together to one big function with plotting of boroughs and venues:

In [174]:
ALL_VENUES = df_venues_raw.copy()
LONDON_GEOJSON = ldn_geojson
LONDON = [51.5074, -0.1278]

def recommend_and_plot(df_groups, df_rent, rent_range=None, acm_type=None, ranking=None, n=5):
    """
    Recommends top n venues and plots the results on the map
    
    """
    available_boroughs = filter_rent_data(df=df_rent, categories=acm_type, rent_range=rent_range)
    p = create_preferences(ranking=ranking)
    df_grp_filtered = df_groups.loc[df_groups['Borough'].isin(available_boroughs)]
    rec_boroughs, df_rec = recommend_boroughs(W=df_grp_filtered, p=p, n_brghs=n)
    
    
    df_rec.reset_index(inplace=True)
    
    df_all = ALL_VENUES.copy()
    df_matched = df_all.loc[(df_all['Borough'].isin(rec_boroughs)) & (df_all['Group'].isin(ranking))]
    
    print('='*80)
    print('= Recommended boroughs based on your preferences')
    print('='*80)
    
    for i, b in enumerate(rec_boroughs):
        print(f"#{i+1}: {b}")
    print('='*80)
    print('= Plotting venues...')
                                                                     
                             
    # Plotting
    map_rec = folium.Map(location=LONDON, tiles='cartodbpositron', zoom_start=10)

    # Chloropleth Map of boundaries, where shading is dependent on median rent for 'All categories'
    folium.Choropleth(
        geo_data=LONDON_GEOJSON,
        fill_color='BuPu',
        data=df_rec,
        columns=['Borough', 'Match'],
        key_on='feature.properties.name',
        weight=1,
        line_color='black',
        fill_opacity=0,
        line_opacity=0.5,
        highlight=True,
    ).add_to(map_rec)  

    df_boroughs = df_matched[['Borough', 'BoroughLat', 'BoroughLon']].drop_duplicates()
                             
    # Add borough markers to map
    for i, row in df_boroughs.iterrows():
        borough = row['Borough']
        coords = [row['BoroughLat'], row['BoroughLon']]
        rent_pref = f"{rent_range[0]} - {rent_range[1]}"
        lbl_str = f"{borough} ({rent_pref}\\xA3)" # create a label with borough name and rent range

        label = folium.Popup(lbl_str, parse_html=True) # so that the label is used as a Popup

        # Add the marker
        folium.CircleMarker(
            coords,
            radius=6,
            popup=label,
            color='black',
            weight=1,
            fill=True,
            fill_color='black',
            fill_opacity=1.0,
            parse_html=False,
        ).add_to(map_rec)  

    # Create random colors:
    from random import randint
    
    color_map = {
        
        'Eating out': '#e41a1c',
        'Entertainment': '#377eb8',
        'Going out': '#ffff33',
        'Green spaces': '#984ea3',
        'Groceries': '#ff7f00',
        'Health and Sports': '#4daf4a',
        'Other': '#999999',
        'Public Transport': '#a65628' ,
        'Shopping': '#f781bf',
    }

    for i, row in df_matched.iterrows():
        # let's plot only every 5th point
        if i % 10 == 0:
            continue
        lat_i = row['Venue Latitude']
        lon_i = row['Venue Longitude']
        venue = row['Venue']
        group = row['Group']

        label = f"{venue} ({group})"

        coords = [lat_i, lon_i]
        folium.CircleMarker(
            coords,
            radius=4,
            popup=label,
            color='black',
            weight=1,
            fill=True,
            fill_color=color_map[group],
            fill_opacity=0.9,
            parse_html=False).add_to(map_rec)

    

    return map_rec
    

In [185]:
rent_range = [400, 800]
acm_type = ['Studio', 'One Bedroom']
group_rank = [
    'Green spaces',
    'Groceries', 
    'Public Transport',
    'Shopping',        
     
]

In [186]:
recommend_and_plot(df_groups=df_groups_norm,
                   df_rent=df_rent,
                   rent_range=rent_range,
                   acm_type=acm_type,
                   ranking=group_rank,
                   n=5)

= Recommended boroughs based on your preferences
#1: Sutton
#2: Barking and Dagenham
#3: Havering
#4: Kingston upon Thames
#5: Enfield
= Plotting venues...
